In [1]:
import requests
import pandas as pd

In [2]:
industry = {
    ("CS000000", "외식업"): [("CS100006", "패스트푸드"), ("CS100010", "커피-음료")],
    ("CS300000", "소매업"): [("CS300002", "편의접"), ("CS300042", "주유소")],
}

category_info = [
    # 점포수
    ("https://golmok.seoul.go.kr/region/selectStoreCount.json",
     "store",
    ["행정구역", "행정구역코드", "구분", "2020 전체 점포수", "2020 프렌차이즈 점포수", "2020 일반 점포수", "2021 전체 점포수", "2021 프렌차이즈 점포수", "2021 일반 점포수", "2022 전체 점포수", "2022 프렌차이즈 점포수", "2022 일반 점포수"]),
    
    # 신생기업 생존율
    ("https://golmok.seoul.go.kr/region/selectSurvivalRate.json",
     "survival",
    ["행정구역코드", "행정구역", "구분", "2020 신생기업 1년 생존율", "2020 신생기업 3년 생존율", "2020 신생기업 5년 생존율", "2021 신생기업 1년 생존율", "2021 신생기업 3년 생존율", "2021 신생기업 5년 생존율", "2022 신생기업 1년 생존율", "2022 신생기업 3년 생존율", "2022 신생기업 5년 생존율"]),
    
    # 연차별 생존율
    ("https://golmok.seoul.go.kr/region/selectSurvivalRate2.json",
     "survival2",
    ["행정구역코드", "행정구역", "구분", "2020 1년 생존율", "2020 3년 생존율", "2020 5년 생존율", "2021 1년 생존율", "2021 3년 생존율", "2021 5년 생존율", "2022 1년 생존율", "2022 3년 생존율", "2022 5년 생존율"]),
    
    # 평균영업기간
    ("https://golmok.seoul.go.kr/region/selectSurvivalAvg.json",
     "operatingPeriod",
    ["행정구역", "행정구역코드", "구분", "2020 최근 10년기준", "2020 최근 30년기준", "2021 최근 10년기준", "2021 최근 30년기준", "2022 최근 10년기준", "2022 최근 30년기준"]),
    
    # 개폐업수(률)
    ("https://golmok.seoul.go.kr/region/selectOpening.json",
     "opening",
    ["행정구역", "행정구역코드", "구분", "2020 개업수", "2020 폐업수", "2020 개업률", "2020 폐업률", "2021 개업수", "2021 폐업수", "2021 개업률", "2021 폐업률", "2022 개업수", "2022 폐업수", "2022 개업률", "2022 폐업률"]),
]

In [3]:
is_first = True

for url, category, columns in category_info:
    for industry_large, industry_large_name in industry.keys():
        for industry_medium, industry_medium_name in industry[(industry_large, industry_large_name)]:
            print(url, category, industry_large, industry_large_name, industry_medium, industry_medium_name)

            params = {
                "stdrYyCd": "2022",
                "stdrSlctQu": "sameQu",
                "stdrQuCd": "4",
                "stdrMnCd": "202212",
                "selectTerm": "quarter",
                "svcIndutyCdL": industry_large,
                "svcIndutyCdM": industry_medium,
                "stdrSigngu": "11",
                "selectInduty": "1",
                "infoCategory": category
            }

            tmp_data = requests.post(url,params=params,headers={'User-Agent':'Mozilla/5.0'}).json()
            
            if len(tmp_data[0].keys()) == len(columns):
                tmp_df = pd.DataFrame([list(i.values()) for i in tmp_data])
                tmp_df.columns = columns[:3] + [" ".join([c, industry_large_name, industry_medium_name]) for c in columns[3:]]
            else:
                tmp_columns = [k for k in tmp_data[0].keys() if len(k.split("_")) < 3]
                tmp_df = pd.DataFrame([list(i.values()) for i in tmp_data], columns=tmp_data[0])[tmp_columns]
                tmp_df.columns = columns[:3] + [" ".join([c, industry_large_name, industry_medium_name]) for c in columns[3:]]
                
            if is_first:
                df = tmp_df.copy()
                is_first = False
            else:
                df = pd.merge(df, tmp_df, on=["행정구역", "행정구역코드", "구분"])

https://golmok.seoul.go.kr/region/selectStoreCount.json store CS000000 외식업 CS100006 패스트푸드
https://golmok.seoul.go.kr/region/selectStoreCount.json store CS000000 외식업 CS100010 커피-음료
https://golmok.seoul.go.kr/region/selectStoreCount.json store CS300000 소매업 CS300002 편의접
https://golmok.seoul.go.kr/region/selectStoreCount.json store CS300000 소매업 CS300042 주유소
https://golmok.seoul.go.kr/region/selectSurvivalRate.json survival CS000000 외식업 CS100006 패스트푸드
https://golmok.seoul.go.kr/region/selectSurvivalRate.json survival CS000000 외식업 CS100010 커피-음료
https://golmok.seoul.go.kr/region/selectSurvivalRate.json survival CS300000 소매업 CS300002 편의접
https://golmok.seoul.go.kr/region/selectSurvivalRate.json survival CS300000 소매업 CS300042 주유소
https://golmok.seoul.go.kr/region/selectSurvivalRate2.json survival2 CS000000 외식업 CS100006 패스트푸드
https://golmok.seoul.go.kr/region/selectSurvivalRate2.json survival2 CS000000 외식업 CS100010 커피-음료
https://golmok.seoul.go.kr/region/selectSurvivalRate2.json survival2 CS300

In [4]:
category_info = [
    # 소득/가구수
    ("https://golmok.seoul.go.kr/region/selectIncome.json",
     "income",
    ["행정구역", "행정구역코드", "구분", "2020 소득분위", "2020 가구수", "2021 소득분위", "2021 가구수", "2022 소득분위", "2022 가구수"]),
    
    # 임대시세
    ("https://golmok.seoul.go.kr/region/selectRentalPrice.json",
     "rent",
    ["행정구역코드", "행정구역", "구분", "2020 환산임대료 전체", "2020 환산임대료 1층", "2020 환산임대료 1층 외", "2021 환산임대료 전체", "2021 환산임대료 1층", "2021 환산임대료 1층 외", "2022 환산임대료 전체", "2022 환산임대료 1층", "2022 환산임대료 1층 외"]),
]

In [5]:
is_first = True

for url, category, columns in category_info:
    print(url, category)
    
    params = {
        "stdrYyCd": "2022",
        "stdrSlctQu": "sameQu",
        "stdrQuCd": "4",
        "stdrMnCd": "202212",
        "selectTerm": "quarter",
        "stdrSigngu": "11",
        "selectInduty": "1",
        "infoCategory": category
    }

    tmp_data = requests.post(url,params=params,headers={'User-Agent':'Mozilla/5.0'}).json()

    tmp_df = pd.DataFrame([list(i.values()) for i in tmp_data])
    tmp_df.columns = columns[:3] + [" ".join([c]) for c in columns[3:]]

    df = pd.merge(df, tmp_df, on=["행정구역", "행정구역코드", "구분"])

https://golmok.seoul.go.kr/region/selectIncome.json income
https://golmok.seoul.go.kr/region/selectRentalPrice.json rent


In [6]:
df.head()

,행정구역,행정구역코드,구분,2020 전체 점포수 외식업 패스트푸드,2020 프렌차이즈 점포수 외식업 패스트푸드,2020 일반 점포수 외식업 패스트푸드,2021 전체 점포수 외식업 패스트푸드,2021 프렌차이즈 점포수 외식업 패스트푸드,2021 일반 점포수 외식업 패스트푸드,2022 전체 점포수 외식업 패스트푸드,...,2022 가구수,2020 환산임대료 전체,2020 환산임대료 1층,2020 환산임대료 1층 외,2021 환산임대료 전체,2021 환산임대료 1층,2021 환산임대료 1층 외,2022 환산임대료 전체,2022 환산임대료 1층,2022 환산임대료 1층 외
0,서울시 전체,11,si,5492,2709,2783,5718,2905,2813,5770,...,4412316,140213,146518,153155,104417,111382,126700,122006,128541,139927
1,종로구,11110,gu,207,122,85,200,115,85,195,...,74942,199091,210992,224889,134928,144030,166811,167010,177511,195850
2,청운효자동,11110515,dong,8,5,3,5,3,2,6,...,5352,150627,182188,224849,96136,82669,86744,123382,132428,155797
3,사직동,11110530,dong,29,19,10,28,18,10,27,...,4758,195313,187502,246659,108733,111134,100447,152023,149318,173553
4,삼청동,11110540,dong,7,3,4,6,2,4,5,...,1335,192051,188341,200614,76391,148249,135664,134221,168295,168139


# 시군구 붙이기

In [7]:
df2 = df[df['행정구역코드'].apply(lambda x: True if len(x)==8 else False)].reset_index(drop=True)
for index,series in df2.iterrows():
    df2.loc[index,'시군구'] = df[df['행정구역코드']==series['행정구역코드'][:5]]['행정구역'].values[0]
df2.insert(0, '시군구', df2.pop('시군구'))
df2.drop(['행정구역코드','구분'],axis=1,inplace=True)

# 값 순위

In [8]:
ranked = df2.set_index(['시군구','행정구역']).groupby('시군구').rank(method='dense',ascending=False).reset_index()

In [9]:
ranked = ranked.rename(columns={column:column+' 구내 순위' for column in ranked.columns if column not in ['시군구','행정구역']})

# 변화량 순위

In [10]:
change = df2[[column for column in df2.columns if '소득' not in column]]

In [11]:
change = change.replace('-',0).replace(' ',0)

In [12]:
change.iloc[:,2:] = change.iloc[:,2:].astype(float)

/tmp/ipykernel_2615/3022397817.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  change.iloc[:,2:] = change.iloc[:,2:].astype(float)


In [ ]:
for column in change.columns:
    if '2022' in column:
        change[column+' 작년 동분기 대비 상승률'] = round(100*(change[column]-change[column.replace('2022','2021')])/change[column.replace('2022','2021')],1)
        change[column+' 작년 동분기 대비 하락률'] = round(100*(change[column.replace('2022','2021')]-change[column])/change[column.replace('2022','2021')],1)


In [14]:
change_ranked = change[['시군구','행정구역']+[column for column in change.columns if column[-1]=='률']].fillna(0).set_index(['시군구','행정구역']).groupby('시군구').rank(method='dense',ascending=False).reset_index()

In [15]:
change_ranked = change_ranked.rename(columns={column:column+' 구내 순위' for column in change_ranked.columns if column not in ['시군구','행정구역']})

# Merge

In [16]:
merged = df2.merge(ranked).merge(change_ranked)

In [17]:
merged = merged.astype({col: int for col in merged.select_dtypes(include='float64').columns})

In [18]:
merged

,시군구,행정구역,2020 전체 점포수 외식업 패스트푸드,2020 프렌차이즈 점포수 외식업 패스트푸드,2020 일반 점포수 외식업 패스트푸드,2021 전체 점포수 외식업 패스트푸드,2021 프렌차이즈 점포수 외식업 패스트푸드,2021 일반 점포수 외식업 패스트푸드,2022 전체 점포수 외식업 패스트푸드,2022 프렌차이즈 점포수 외식업 패스트푸드,...,2022 폐업률 소매업 주유소 작년 동분기 대비 상승률 구내 순위,2022 폐업률 소매업 주유소 작년 동분기 대비 하락률 구내 순위,2022 가구수 작년 동분기 대비 상승률 구내 순위,2022 가구수 작년 동분기 대비 하락률 구내 순위,2022 환산임대료 전체 작년 동분기 대비 상승률 구내 순위,2022 환산임대료 전체 작년 동분기 대비 하락률 구내 순위,2022 환산임대료 1층 작년 동분기 대비 상승률 구내 순위,2022 환산임대료 1층 작년 동분기 대비 하락률 구내 순위,2022 환산임대료 1층 외 작년 동분기 대비 상승률 구내 순위,2022 환산임대료 1층 외 작년 동분기 대비 하락률 구내 순위
0,종로구,청운효자동,8,5,3,5,3,2,6,4,...,1,1,1,1,9,8,2,16,2,15
1,종로구,사직동,29,19,10,28,18,10,27,16,...,1,1,1,1,7,10,8,10,4,13
2,종로구,삼청동,7,3,4,6,2,4,5,2,...,1,1,1,1,2,15,13,5,10,7
3,종로구,부암동,11,7,4,11,7,4,10,7,...,1,1,1,1,14,3,16,2,8,9
4,종로구,평창동,3,1,2,3,1,2,4,2,...,1,1,1,1,16,1,9,9,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,강동구,성내2동,9,6,3,10,8,2,9,7,...,1,2,1,1,14,5,5,13,17,2
420,강동구,성내3동,19,9,10,22,11,11,22,11,...,1,2,1,1,13,6,14,4,7,12
421,강동구,길동,24,14,10,23,13,10,27,18,...,1,2,1,1,10,9,9,9,11,8
422,강동구,둔촌1동,1,1,0,1,1,0,1,1,...,1,2,1,1,18,1,16,2,16,3
